<a href="https://colab.research.google.com/github/Karthi-2-2/PROJECT/blob/main/Welcome_To_Colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Welcome to Colab!

In [1]:
# ============================================
# 📦 MULTIVARIATE TIME SERIES + TRANSFORMER
# ============================================
import numpy as np
import torch
import torch.nn as nn
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.holtwinters import ExponentialSmoothing

# -------------------------------
# 1️⃣ DATA GENERATION
# -------------------------------
def generate_data(n=1500):
    t = np.arange(n)
    x1 = np.sin(2*np.pi*t/24) + 0.01*t + np.random.normal(0,0.2,n)
    x2 = np.cos(2*np.pi*t/12) + 0.5*np.sin(2*np.pi*t/48) + np.random.normal(0,0.2,n)
    return np.vstack([x1, x2]).T

data = generate_data()
scaler = MinMaxScaler()
data = scaler.fit_transform(data)

# -------------------------------
# 2️⃣ WINDOWING
# -------------------------------
def make_seq(data, win=30):
    X, y = [], []
    for i in range(len(data)-win):
        X.append(data[i:i+win])
        y.append(data[i+win])
    return torch.tensor(X).float(), torch.tensor(y).float()

X, y = make_seq(data)
X_train, y_train = X[:1100], y[:1100]
X_test, y_test = X[1100:], y[1100:]

# -------------------------------
# 3️⃣ TRANSFORMER MODEL
# -------------------------------
class Transformer(nn.Module):
    def __init__(self):
        super().__init__()
        self.enc = nn.TransformerEncoder(
            nn.TransformerEncoderLayer(d_model=2, nhead=1), num_layers=2
        )
        self.fc = nn.Linear(2, 2)

    def forward(self, x):
        x = self.enc(x)
        return self.fc(x[:, -1])

model = Transformer()
opt = torch.optim.Adam(model.parameters(), lr=0.001)
loss_fn = nn.MSELoss()

# -------------------------------
# 4️⃣ TRAINING
# -------------------------------
for epoch in range(20):
    pred = model(X_train)
    loss = loss_fn(pred, y_train)
    opt.zero_grad(); loss.backward(); opt.step()
    print(f"Epoch {epoch+1}: {loss.item():.4f}")

# -------------------------------
# 5️⃣ EVALUATION
# -------------------------------
with torch.no_grad():
    pred = model(X_test).numpy()
    true = y_test.numpy()

rmse = np.sqrt(mean_squared_error(true, pred))
mae = mean_absolute_error(true, pred)
mape = np.mean(np.abs((true - pred) / true)) * 100
print("\nTransformer → RMSE:", rmse, " MAE:", mae, " MAPE:", mape)

# -------------------------------
# 6️⃣ CLASSICAL MODELS
# -------------------------------
train_series = data[:1100,0]
test_series = data[1100:,0]

arima = ARIMA(train_series, order=(3,1,2)).fit()
arima_pred = arima.forecast(len(test_series))

hw = ExponentialSmoothing(train_series).fit()
hw_pred = hw.forecast(len(test_series))

print("\nARIMA RMSE:", np.sqrt(mean_squared_error(test_series, arima_pred)))
print("HW RMSE:", np.sqrt(mean_squared_error(test_series, hw_pred)))



/tmp/ipython-input-2627663669.py:33: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at /pytorch/torch/csrc/utils/tensor_new.cpp:253.)
  return torch.tensor(X).float(), torch.tensor(y).float()
/usr/local/lib/python3.12/dist-packages/torch/nn/modules/transformer.py:392: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(


Epoch 1: 0.3751
Epoch 2: 0.3714
Epoch 3: 0.3683
Epoch 4: 0.3651
Epoch 5: 0.3612
Epoch 6: 0.3483
Epoch 7: 0.3416
Epoch 8: 0.3358
Epoch 9: 0.3307
Epoch 10: 0.3263
Epoch 11: 0.3205
Epoch 12: 0.3153
Epoch 13: 0.3106
Epoch 14: 0.3059
Epoch 15: 0.3013
Epoch 16: 0.2970
Epoch 17: 0.2924
Epoch 18: 0.2881
Epoch 19: 0.2842
Epoch 20: 0.2794

Transformer → RMSE: 0.6536651750314446  MAE: 0.6323838829994202  MAPE: 111.24264

ARIMA RMSE: 0.18848518947610973
HW RMSE: 0.2012069173447778

🔍 Transformer attention focuses more on recent time steps
and peaks at seasonal lags (≈12 and 24), showing it learned both
short-term dependencies and periodic structure.


/usr/local/lib/python3.12/dist-packages/statsmodels/base/model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
